<a href="https://colab.research.google.com/github/kywch/geo-colab/blob/master/TreePedia_Step2_checking_gsv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**To access the google streetview API, one should have the API key.**
* First, get started with Google Maps platform: https://developers.google.com/maps/gmp-get-started
* Then, get an API key to use Street View Static API: https://developers.google.com/maps/documentation/streetview/get-api-key

---
This notebook uses google-streetview package: https://pypi.org/project/google-streetview/. 
* For documentation, please see https://rrwen.github.io/google_streetview/



In [1]:
!pip install google-streetview

  Created wheel for google-streetview: filename=google_streetview-1.2.9-cp36-none-any.whl size=9781 sha256=854daa0701ca30db232411d258cda611dfabea79aa5eb4e097ef3f9aa33c9fca
  Stored in directory: /root/.cache/pip/wheels/e1/ce/c0/e90d7ee251659d215b7ab21a7cd80cfd65e1720fc3dca3b9ed
  Created wheel for kwconfig: filename=kwconfig-1.1.7-cp36-none-any.whl size=4981 sha256=fba54bc86d7f8a64bae9805cf9e829d94ce397b8a398f6f49731d0cec0d34346
  Stored in directory: /root/.cache/pip/wheels/22/fa/82/fed4c9e2829a6210470b121673adddedf169b6319112b3ba20
Successfully built google-streetview kwconfig


In [84]:
import pandas as pd
import google_streetview.api
import math

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!ls drive/My\ Drive/StreetPsych

Chicago  HydePark  key.txt


In [19]:
# Google Street View Static API key
with open('drive/My Drive/StreetPsych/key.txt') as file:
  api_key = file.read()

In [100]:
location = 'HydePark'
picked_df = pd.read_csv('drive/My Drive/StreetPsych/' + location + '/seed_' + location + '.csv')
picked_df

,Unnamed: 0,lat,lng
0,0,41.792792,-87.581787
1,1,41.792728,-87.581267
2,2,41.792679,-87.580695
3,3,41.792377,-87.580708
4,4,41.788694,-87.584384
...,...,...,...
741,741,41.796734,-87.602767
742,742,41.797259,-87.602779
743,743,41.797784,-87.602792
744,744,41.795585,-87.602745


In [103]:
params = []
for row in picked_df.head(20).iterrows():
  params.append({
            'key': api_key,
            'location': str(row[1]['lat']) + ',' + str(row[1]['lng'])
  })
#'size': '600x480', # max 640x640 pixels
#print(params)

In [104]:
results = google_streetview.api.results(params)
#results.preview()
len(results.metadata)

20

In [105]:
# input - df: a Dataframe, chunkSize: the chunk size
# output - a list of DataFrame
# purpose - splits the DataFrame into smaller chunks
def split_dataframe(df, chunk_size = 200): 
    chunks = list()
    num_chunks = len(df) // chunk_size + 1
    for i in range(num_chunks):
        chunks.append(df[i*chunk_size:(i+1)*chunk_size])
    return chunks

def generate_param(df, api_key):
  params = []
  for row in df.iterrows():
    params.append({
              'key': api_key,
              'location': str(row[1]['lat']) + ',' + str(row[1]['lng'])
    })
  return params

In [106]:
#split_dataframe(picked_df)[1][['lat','lng']].reset_index(drop=True)

In [107]:
combined_df = pd.DataFrame()
split_dfs = split_dataframe(picked_df)
count = 1

for df in split_dfs:
  print('Processing ', count, ' / ', len(split_dfs))
  results = google_streetview.api.results(generate_param(df, api_key))
  gsv_info = []
  for row in results.metadata:
    if 'OK' in row['status']:
      if 'Google' in row['copyright']:
        gsv_info.append([row['location']['lat'], row['location']['lng'], row['pano_id'], row['date'], row['status']])
      else:
        gsv_info.append([math.nan, math.nan, math.nan, math.nan, math.nan])
    else:
      gsv_info.append([math.nan, math.nan, math.nan, math.nan, math.nan])

  combined_df = combined_df.append(
      pd.concat([df[['lat','lng']].reset_index(drop=True), pd.DataFrame(gsv_info, columns = ['gsv_lat', 'gsv_lng', 'pano_id', 'date', 'status'])], axis=1)
  )

  count += 1


Processing  1  /  4
Processing  2  /  4
Processing  3  /  4
Processing  4  /  4


In [108]:
combined_df.reset_index(drop=True).to_csv('drive/My Drive/StreetPsych/' + location + '/gsv_seed_' + location + '.csv')